In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import lightkurve as lk


from flare_timestamps import get_timestamps

Search for the target in the Kepler database. Here the target KIC5695372 was chosen. Then download the lightcurve collection for that target. The collection conatins the lightcurves for all quarters in which the target was observed. The lightcurves are stored in a fits file. 

In [ ]:
search_result= lk.search_lightcurve('KIC 5695372', mission='Kepler', cadence='long') 
search_result
lc_collection = search_result.download_all()
lc_collection



Plot the flux data from all qurters. For all further analysis the PDCSAP flux will be used. This is the flux data after the Pre-Search Data Conditioning (PDC) pipeline which removes instrumental signatures from the data.

In [ ]:
ax= lc_collection[0].plot(column='pdcsap_flux', label=None)
for lc in lc_collection[1:]:
    lc.plot(ax=ax, column='pdcsap_flux', label=None)
plt.title('KIC 6034120: Flux over all quarters ')
plt.ylabel('Flux (e-/s)')
plt.savefig('KIC10011070_flux_over_all_quarters.pdf')
plt.show()

Now all the quarters have to be normalized to the same value. This could be done manually but the lightkurve package has a function for that which normalizes the flux data to the same value.

Now the quartes have to be stitched together in order to get one lightcurve. This is done by the stitch() function of the lightkurve package. The function takes the lightcurve collection as input and returns the stitched lightcurve.

In [ ]:
lc_stitched = lc_collection.stitch()
lc_collection.stitch().plot(color='black');

#plot the peaks to the matched time stamps
plt.title('Stitched lightcurve over all quarters')
plt.ylabel('Normalized Flux (ΔF/F)') 
plt.grid(True)
plt.savefig('KIC5695372_add_flares.pdf')
plt.show()


In order to get the lightcurve for the flare, the time of the flare has to be known. For that the last column of the superflares.txt file is used. This is doen in the get_timestamps() function. The function takes the superflares.txt file as input and returns the timestamps of the flares.

In [ ]:
#do the exact same as the above but only for one target
target_names= np.loadtxt('superflares.txt', dtype='str', usecols=0)
target_name = '5695372'
input_file = 'superflares.txt'
all_timestamps = get_timestamps(target_name, input_file)

try:
    search_result = lk.search_lightcurve(f'KIC {target_name}', mission='Kepler', cadence='long')
    lc_collection = search_result.download_all()
except:
    print(f'Lightcurve for KIC {target_name} not found')
    pass

The unit of the timestamps has to be converted and they are rounded (this is only used for the xticks in the plot)

In [ ]:
timestamps = []
for element in all_timestamps:
    element = float(element) + float(2400000)
    timestamp = np.round(element - 2454833, 2)
    timestamps.append(timestamp)


t= np.array(lc_collection.stitch().time.value)
lc= np.array((lc_collection.stitch().flux) - 1)

Next a loop is created over all flare timestamps for one target (usually one star has multiple recorded superflares). Each time the the time is plotted against the stars flux in an interval 40 days before and 20 days after the flare.

In [ ]:
for element in timestamps:

    plt.figure(figsize=(10, 10))
    plt.plot(t, lc, 'black',  lw=0.3)
    plt.xlim(element-40, element+20)
    plt.xticks([element-40, element-30, element-20, element-10, element, element+10, element+20], ['-40','-30','-20','-10','0','10','20'])
    plt.ylim(-0.01,0.02)
    plt.grid()
    plt.title('Kepler Lightcurve for KIC: ' + target_name)
    # plt.title('Flare date: ' + str(element) + ' days')
    plt.xlabel('Day from peak')
    plt.ylabel('Flux (ΔF/F)')
    plt.show()